In [1]:
import numpy as np

In [2]:
from deap import gp, creator, tools, base, algorithms
from random import randint, seed
from functools import partial

pset = gp.PrimitiveSetTyped('MAIN', 1)
pset.addPrimitive(name='add', arity=2, rettype=float, argtypes=[float, float])
pset.addPrimitive(name='sub', arity=2, rettype=float, argtypes=[float, float])
pset.addPrimitive(name='mul', arity=2, rettype=float, argtypes=[float, float])
pset.addPrimitive(name='div', arity=2, rettype=float, argtypes=[float, float])
pset.addPrimitive(name='pow', arity=2, rettype=float, argtypes=[float, float])
pset.addPrimitive(name='sqrt', arity=1, rettype=float, argtypes=[float])
pset.addPrimitive(name='and', arity=2, rettype=bool, argtypes=[bool, bool])
pset.addPrimitive(name='or', arity=2, rettype=bool, argtypes=[bool, bool])
pset.addPrimitive(name='not', arity=2, rettype=bool, argtypes=[bool])
pset.addPrimitive(name='lt', arity=2, rettype=bool, argtypes=[float, float])
pset.addPrimitive(name='gt', arity=2, rettype=bool, argtypes=[float, float])
pset.addPrimitive(name='eq', arity=2, rettype=bool, argtypes=[float, float])
pset.addEphemeralConstant('Constante', partial(randint, 1, 10))

In [3]:
pset.renameArguments(ARG0='cost', ARG1='area', ARG2='degree', ARG3='original')

In [4]:
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create('Individual', gp.PrimitiveTree, fitness=creator.FitnessMin)

In [5]:
import subprocess
import scipy.stats as st

CAMINHO = r'/home/mpvreal/Code/Faculdade/tcc/llvm/testes/x86-64'

def obter_scripts_benchmark(benchmark: str) -> dict[str, str]:
  """
  Obtém os scripts de compilação e execução do benchmark.
  """
  global CAMINHO
  
  return { 'compile': f'{CAMINHO}/{benchmark}/compile', 'run': f'{CAMINHO}/{benchmark}/runbench' }

def compilar_benchmark(heuristica: str, scripts: dict[str, str]) -> None:
  """
  Escreve função heurística da alocação em um arquivo e compila o benchmark.
  """
  with open('../HeuristicFunction.txt', 'w+') as f:
    f.write(heuristica)

  subprocess.run(scripts['compile'], shell=True)

def coletar_tempo_exec(scripts: dict[str, str]) -> float:
  """
  Executa o benchmark e coleta o tempo de execução.
  """
  tempo = subprocess.run(scripts['run'], 
                         shell=True, 
                         stdout=subprocess.PIPE).stdout.decode('utf-8').replace(',', '.').strip()

  return float(tempo)

def calcular_media_e_erro(dados, confianca=0.95):
  """
  Obtém a média e margem de erro de um conjunto de dados, com um intervalo de confiança de 95%.
  """
  a = 1.0 * np.array(dados)
  n = len(a)

  media, erro_medio = np.mean(a), st.sem(a)
  h = erro_medio * st.t.ppf((1 + confianca) / 2., n-1)

  return media, h

def avaliar_fitness(individual, benchmark: str):
  """
  Escreve a heurística em um arquivo, compila o benchmark e coleta o tempo de execução médio após
  30 rodadas.
  """
  heuristica = str(gp.PrimitiveTree(individual))
  scripts = obter_scripts_benchmark(benchmark)
  compilar_benchmark(heuristica, scripts)

  tempos = []
  for _ in range(30):
    tempos.append(coletar_tempo_exec(scripts))
  
  media, margem_erro = calcular_media_e_erro(tempos)

  return media,

In [6]:
toolbox = base.Toolbox()
toolbox.register('expr', gp.genHalfAndHalf, pset=pset, min_=1, max_=5)
toolbox.register('individual', tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register('population', tools.initRepeat, list, toolbox.individual)
toolbox.register('evaluate', avaliar_fitness, benchmark='519.lbm_r')
toolbox.register('select', tools.selTournament, tournsize=4)
toolbox.register('mate', gp.cxOnePoint)
toolbox.register('expr_mut', gp.genFull, min_=0, max_=2)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate('mate', gp.staticLimit(key=attrgetter('height'), max_value=17))
toolbox.decorate('mutate', gp.staticLimit(key=attrgetter('height'), max_value=17))

In [7]:
seed(318)

pop = toolbox.population(n=300)
hof = tools.HallOfFame(3)         # Melhores indivíduos ao final do treinamento

stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
stats_size = tools.Statistics(len)
mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
mstats.register('avg', np.mean)
mstats.register('std', np.std)
mstats.register('min', np.min)
mstats.register('max', np.max)

pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                halloffame=hof, verbose=True)

function@LBM_allocateGrid,0,0,0,0,0,1,0,0,0,0,1
function@LBM_freeGrid,0,0,0,0,0,1,0,0,0,0,1
function@LBM_initializeGrid,0,0,0,0,0,0,0,0,0,0,0
function@LBM_swapGrids,0,0,0,0,0,0,0,0,0,0,0
function@LBM_loadObstacleFile,0,0,0,0,0,9,0,0,0,0,33860.5
function@LBM_initializeSpecialCellsForLDC,0,0,0,0,0,7,0,0,0,0,3740.12
function@LBM_initializeSpecialCellsForChannel,0,0,0,0,0,2,0,0,0,0,10954.5
function@LBM_performStreamCollideBGK,2,2,0,4,0,3,15.75,15.75,63.5,0,47.625
function@LBM_performStreamCollideTRT,29,31,0,51,0,9,460.375,492.125,809.625,0,158.875
function@LBM_handleInOutFlow,5,11,0,14,0,5,159.375,350.625,446.25,0,159.375
function@LBM_showGridStatistics,21,2,0,14,0,6,149.75,31.875,221,0,19.9375
function@LBM_storeVelocityField,3,0,0,3,0,12,1088,0,1088,0,114694
function@LBM_compareVelocityField,22,1,0,23,0,15,623680,32768,656448,0,139302
function@main,0,0,0,0,0,14,0,0,0,0,54.1667
function@MAIN_parseCommandLine,0,0,0,0,0,3,0,0,0,0,3
function@MAIN_printInfo,0,0,0,0,0,0,0,0,0,0,0
function@MAIN_

KeyboardInterrupt: 